# Current clamp
Current clamp recordings are one of the two most common experiment types you will see in papers using path-clamp. Current clamp experiments allow you to get active parameters of cells. Namely gain, rheobase and spike threshold. Current clamp recordings typically use stepped currrent injections or ramps and you are investigating the voltage change due to the current injection and whether the cell spikes.

## Experimental considerations for current clamp recordings
### Internal and external solutions
You will need to a standard ACSF for recording. Usually the ACSF bath is mantained at physiological temperature which is considered ~32C however you can record at room temperature. I recommend recording at 32C. You can include drugs in the bath to block synaptic activity but do not need to include these. If you are having a lot of recurrent input that is driving spikes, depolarizations or hyperpolarizations in your cells if may be good to include some drugs to block synpatic currents. The one problem with blocking synaptic currents if that cells change their functionality due to changes in or lack of synaptic input. These are considerations you need to consider. I would recommend starting without any drugs in the bath to keep it simple.

### To inject current a holding current or not
One thing to consider is whether you should inject a holding current to bring the cell to a specific resting potential value. Many studies inject some current to hold the cell at around -70mV and run their current pulses or ramps. There are a few studies that do not inject any holding current and even argue against injecting any holding current. Both methods have their merits.

### Steps vs ramps
As mentioned in the beginning, you can record stepped currents or ramps. Stepped currents are the most common. Usually there are negative and positive steps. The negative steps usually go as low as -150 to -200 pA and positive currents can as high as 600 pA (some interneurons have a very high rheobase). One problem with ramps is that some cells have depolarization block and may not spike during a ramp but will during a set of stepped pulses. Depolarization block is the inactivation of voltage-gated sodium channels due to long membrane depolarizations that does not elicit spikes. There are conditions, such as epilepsy, where depolarization block may actually be a feature of the altered cell physiology and you could use the ramp to test whether is still occurs.

## Analyzing current clamp data
Below is a tutorial on analyzing current clamp data with a specific focus on current injections. The tutorial go over what features are import in the waveforms and how to analyze the extracted data.

First we are going to import some python packages.

In [ ]:
import json
import urllib

import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, CustomJS, Slider, Spinner, Span
from bokeh.plotting import figure
from scipy import optimize, signal, stats

output_notebook()

Next we are going to load the data. All the data is stored on json files. While this file type is not the most practical for storing electrophysiological data, it is the very convenient since it does not require any third party python packages.

In [ ]:
temp_path = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/current_clamp/"
exp_dict = {}
for index in range(1, 69):
    url = urllib.request.urlopen(temp_path + f"{index}.json")
    temp = json.load(url)
    exp_dict[index] = temp
x_array = np.arange(len(exp_dict[1]["array"])) / 10

The first thing to do is look through your data just to see what it looks like. For reference the data in this tutorial is from a layer 5 cell in the ACC of a P16 mouse. 
- The recorded data is usually in mV, as is the case for this data.
- There is a short baseline of about 300 ms before the current injection starts.
- There are current injections that make the voltage go negative and ones that the cell goes positive.
- There is a point where the positive current injections make the cell spike.
- The current injection is finite but also not short enough that only one spike is ever evoked. This is important for calculating the FI curve
- There are 4 cycles where the pulse amplitude start at -100 pA and is increased in 25 pA steps until 300 pA is reached.

In [ ]:
# Initial data
source = ColumnDataSource(data={"x": np.arange(20000) / 10, "y": exp_dict[1]["array"]})

# Create a plot
plot = figure(
    x_axis_label="Time (ms)", y_axis_label="Voltage (mV)", height=250, width=400
)
plot.line("x", "y", source=source, line_color="black")
spinner = Spinner(title="Acquisition", low=1, high=68, step=1, value=1, width=80)

# JavaScript callback to fetch JSON data and update plot
callback = CustomJS(
    args=dict(source=source, spinner=spinner),
    code="""
    let val = spinner.value
    let URL = `https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/current_clamp/${val}.json`
    fetch(URL)
    .then(response => response.json())
    .then(data => {
        source.data.y = data["array"];
        source.change.emit();
    })
    .catch(error => console.error('Error fetching data:', error));
""",
)

# Add a button to trigger the callback
spinner.js_on_change("value", callback)

# Layout and show
layout = column(spinner, plot)
show(layout)

First, we will define some important features of the acquisition so that we can reuse the settings throughout the analysis. It is important to note the all the parameters are going to be in samples. The current files were recorded at 10000 Hz so we are multiply the time by 10.

In [ ]:
baseline_start = 0
baseline_end = 3000
pulse_start = 3000
pulse_end = 10000

## Measuring delta V and getting the IV curve
The first concept we are going to cover is delta V and analyzing the IV (current-voltage curve). Delta V is simply the change in voltage due to the current injection. The IV curve is used to calculate the membrane resistance. Remember that voltage difference = current * resistance ($\Delta V=IR$). This can be calculated by finding the baseline voltage of the acquisition and subtracting voltage during the pulse. There are some things consider when calculating the voltage during the current injection. There can be changes in voltage due channels that temporarily open such as due to Ih channels. To avoid the contribution of these channels I recommend get the mean voltage of the current injection in the last 50%, or even 30%, of the current injection.

In [ ]:
acq = exp_dict[1]["array"]
x = np.arange(20000) / 10

# We define the baseline as the mean of the acquisiton from 0 to 300 ms
baseline_v = np.mean(acq[baseline_start:baseline_end])

# We define the pulse voltage as the mean of the acquisition for the last 50% of the current injection
p50 = ((pulse_end - pulse_start) // 2) + pulse_start
injection_v = np.mean(acq[p50:pulse_end])
delta_v = injection_v - baseline_v

plot = figure(x_axis_label="Time (ms)", y_axis_label="Voltage (mV)")
plot.line(x, acq, line_color="black", legend_label="Acquisition")
plot.line(
    x[baseline_start:baseline_end],
    acq[baseline_start:baseline_end],
    line_color="#37f2fc",
    legend_label="Baseline",
)
plot.line(
    np.arange(p50, pulse_end) / 10,
    acq[p50:pulse_end],
    line_color="#fcba37",
    legend_label="Pulse",
)
plot.line(
    [p50 / 10, p50 / 10],
    [baseline_v, injection_v],
    line_color="#fc37fc",
    legend_label="Delta V",
)
show(plot)

Next we will need to measure the delta V for all pulses except those with spikes. The reason that we do not measure the delta V for acquisitions with spikes is that spiking activity is a seperate state compared to non-spiking. So we will simply exclude acquisitions with spikes. We will do this by ignoring acquisitions with voltages greater than -20 mV. We will also need the pulse amplitude of each acquisition which is located in the file.

In [ ]:
p50 = ((pulse_end - pulse_start) // 2) + pulse_start
delta_v = []
current_amplitude = []
for acq_num, acq in exp_dict.items():
    voltages = acq["array"]
    if np.max(voltages) < -20:
        current_amplitude.append(acq["pulse_amp"])
        baseline_v = np.mean(voltages[baseline_start:baseline_end])
        injection_v = np.mean(voltages[p50:pulse_end])
        delta_v.append(injection_v - baseline_v)
plot = figure(x_axis_label="Current (pA)", y_axis_label="Delta V (mV)")
plot.scatter(current_amplitude, delta_v, line_color="black", color="grey", size=10)
show(plot)

In the plot above you will notice there is a linear relationship between current and voltage. To get the membrane resistance from these data we just have to run a linear regression between the current and delta V. There are a couple ways that you can run the linear regression. One is you can take all delta Vs, you can take just a subset or you can rectify or take the absolute value of the delta Vs before running the regression. We will do all three since it is easy to do in Python. One really important thing to note is the units of the regressors. current is in pA and delta V is in mV and we need to get to MOhms. To do this we multiply the slope by 1000 to get MOhm. If you run the code below you see that the answers are fairly close. One reason to choose the subset version is that you may change the membrane resistance as the cell gets close to spiking even if it does not spike. However, all three calculations are valid.

In [ ]:
mem_res_all = stats.linregress(current_amplitude, delta_v)
subset = np.where(np.array(current_amplitude) <= 50)[0]
mem_res_subset = stats.linregress(
    np.array(current_amplitude)[subset], np.array(delta_v)[subset]
)
mem_res_rectified = stats.linregress(np.abs(current_amplitude), np.abs(delta_v))

# \n is just added to print on separate lines
print(
    f"All values: {mem_res_all.slope * 1000}",
    f"Subset: {mem_res_subset.slope * 1000}",
    f"Rectified: {mem_res_rectified.slope * 1000}",
    sep="\n",
)
x_fit = np.linspace(min(current_amplitude), max(current_amplitude), num=50)
plot = figure(x_axis_label="Current (pA)", y_axis_label="Delta V (mV)")
plot.scatter(
    current_amplitude, delta_v, line_color="black", color="grey", size=10, alpha=0.6
)
plot.line(
    x_fit,
    mem_res_all.intercept + x_fit * mem_res_all.slope,
    legend_label="Fit all",
    line_color="#fcba37",
    line_width=2,
)
plot.line(
    x_fit,
    mem_res_subset.intercept + x_fit * mem_res_subset.slope,
    legend_label="Fit subset",
    line_color="#fc37fc",
    line_width=2,
)
plot.line(
    x_fit,
    mem_res_rectified.intercept + x_fit * mem_res_rectified.slope,
    legend_label="Fit rectified",
    line_color="#37f2fc",
    line_width=2,
)
plot.legend.location = "top_left"
show(plot)

## Ih voltage sag
Hyperpolarization-activated cyclic nucleotide–gated (HCN) channels are typically responsible for the voltage sag in a current clamp acquisitions. Voltage sag is when the voltage drop is initially larger at the beginning of the current injection than at the end. Many cell types have Ih voltage sag such as dopaminergic cells and layer 5 pyramidal neurons. Measuring voltage sag is fairly straightforward. You just measure the voltage difference between the peak sag and and the voltage in the second half of the current injection similar to measuring delta V. The main way I have seen voltage sag reported is by measuring the sag on the most negative current injection. You could also technically run an IV curve for the voltage sag. To truly determine whether the voltage sag is due to HCN channels you would need to do a flow-in experiment with the drug ZD7288. If you want to run the IV curve I challenge you to modify the IV code above to get the resistance of the channels contributing to the voltage sag.

In [ ]:
p50 = ((pulse_end - pulse_start) // 2) + pulse_start
delta_v = []
current_amplitude = []
sag_loc = []
acqs = []
for acq_num, acq in exp_dict.items():
    voltages = acq["array"]
    if acq["pulse_amp"] == -100:
        acqs.append(voltages)
        current_amplitude.append(acq["pulse_amp"])
        sag_v = np.min(voltages[pulse_start:p50])
        sag_loc.append(np.argmin(voltages[pulse_start:p50]) + pulse_start)
        injection_v = np.mean(voltages[p50:pulse_end])
        delta_v.append(sag_v - injection_v)
print(f"Voltage sag: {np.mean(delta_v):.3f} mV")

Let's confirm that the voltage sag was correctly measured. Note that when we collected the sag location above we collected the sample number so that needs to be converted to time in ms by dividing by 10.

In [ ]:
plots = []
for sloc, dv, vs in zip(sag_loc, delta_v, acqs):
    plot = figure(height=150, width=300)
    plot.line(x_array, vs, line_color="black")
    plot.line(
        [sloc / 10, sloc / 10],
        [vs[sloc], vs[sloc] - dv],
        line_color="#fc37fc",
        line_width=3,
    )
    plot.axis.visible = False
    plot.grid.visible = False
    plot.outline_line_color = None
    plots.append(plot)
grid = gridplot([plots[:2], plots[2:]])
show(grid)

## Rheobase
Rheobase is the minimum current required to get a cell to spike. This measure is directly related to membrane resistance through the equation: V=IR. When resistence increases, the current need to achieve the same delta V decreases. This means that a cell will higher membrane resistance will likely need less synaptic to be able to spike. There are a couple ways you can find rheobase. Find the minimum current needed to get the cell to spike out of all 4 cycles. I do not recommend this since there is variability between cycles and the variability could be affected by a treatment or other factors. Find the minimum current needed to get the cell to spike for each cycle and average the result. This is the method I recommend. The way I find rheobase below depends on the acquisitions being assigned to a cycle (1 to inf) which ClampSuite automatically. This may not be the case for your data. There are other ways to find rheobase


In [ ]:
rheobase = []
rheobase_acq = []
for index in range(2, len(exp_dict) + 1):
    if (np.max(exp_dict[index]["array"]) > 30) & (
        np.max(exp_dict[index - 1]["array"]) < 30
    ):
        rheobase.append(exp_dict[index]["pulse_amp"])
        rheobase_acq.append(index)
        exp_dict[index]["rheobase"] = True
        exp_dict[index-1]["rheobase"] = False
print(f"Rheobase: {np.mean(rheobase)} pA")

Let's confirm that we have the right rheobase values by plotting the rheobase acquistion in red and the previous acquisition in black.

In [ ]:
plots = []
for acq in rheobase_acq:
    plot = figure(height=150, width=300)
    plot.line(
        x_array,
        exp_dict[acq - 1]["array"],
        line_color="black",
    )
    plot.line(
        x_array,
        exp_dict[acq]["array"],
        line_color="red",
    )
    plot.axis.visible = False
    plot.grid.visible = False
    plot.outline_line_color = None
    plots.append(plot)
grid = gridplot([plots[:2], plots[2:]])
show(grid)

## Spike frequency
Spike frequency is used to create the FI curve. The FI curve relates several pieces of information such as rheobase, gain and maximum firing rate. The firing rate is calculated as the number of spikes that occur during the current injection or you can derive the firing rate from the inter-event (spike) interval (IEI or ISI). The IEI/ISI requires that you have at least two spikes in the acquisition. IEI/ISI is also useful because you can derive the firing rate variability within an acquisition. Three important features you can get from the FI curve that you will need to curve fit to get.
- Slope/gain: The input gain of a cell
- Maximum firing rate: The response gain of a cell
- Current offset
- The voltage offset: This is usually close to zero

If you want to learn more about gain I would recommend looking at Ferguson and Cardin, 2020 {cite}`ferguson_mechanisms_2020`. The Sigmoid curve and gain are explained in-depth below. If you are using the Jupyter Notebook version of the tutorial there is an interactive widget that allows you change the input and output gain of a the Sigmoic curve which can help understand how the different parameters change the curve.


### Sigmoid fit the FI curve to get the gain and max firing rate
Many papers will just analyze their FI data using a repeated measures ANOVA. However, this method only tells whether or not the FI curves are different and where they are different. To get a better idea of why the curves are different you really need to fit a sigmoid function data for each cell. This will get you the gain of a cell or the slope of the FI curve as well as the estimated maximum firing rate and the current offset. This is a simple example of how to do this in Python.

### Understanding of gain
Below is an example of how changes in gain would work. Note that to get a input gain difference without a rheobase difference the midpoint current has to be changed. That means to test whether there is a gain difference you will like nead to run a multivariate regression or ANOVA because these two factors are not independent.

In [ ]:
def sigmoid(x, max_value, midpoint, slope, offset):
    return 1 / (1 + np.exp((x - midpoint) / slope)) * max_value + offset


x = np.linspace(0, 400)
plot1 = figure(height=300, width=250, title="Input gain difference")
plot1.line(x, sigmoid(x, 17, 200, -40, 0))
plot1.line(x, sigmoid(x, 17, 170, -30, 0), color="orange")
plot2 = figure(height=300, width=250, title="Response gain difference")
plot2.line(x, sigmoid(x, 20, 200, -40, 0))
plot2.line(x, sigmoid(x, 17, 200, -40, 0), color="orange")
plot3 = figure(height=300, width=250, title="Rheobase difference")
plot3.line(x, sigmoid(x, 17, 200, -40, 0))
plot3.line(x, sigmoid(x, 17, 150, -40, 0), color="orange")
grid = gridplot([[plot1, plot2, plot3]])
show(grid)

Below is a simple interactive plot where you can play arount with the slope, max value and the midpoint of two sigmoid curves to get an idea of how each of the factors.

In [ ]:
def sigmoid(x, max_value, midpoint, slope, offset):
    return 1 / (1 + np.exp((x - midpoint) / slope)) * max_value + offset


x = np.linspace(0, 400)
y1 = sigmoid(x, 16.0, 200.0, -40.0, 0)
y2 = sigmoid(x, 16.0, 200.0, -40.0, 0)

source1 = ColumnDataSource({"x": np.linspace(0, 400), "y1": y1, "y2": y2})
source2 = ColumnDataSource(
    {"x": np.linspace(0, 400, num=49), "y1": np.diff(y1), "y2": np.diff(y2)}
)

plot1 = figure(width=400, height=400)
plot1.line("x", "y1", source=source1, line_width=3, line_alpha=0.6)
plot1.line("x", "y2", source=source1, line_width=3, line_alpha=0.6, line_color="orange")
plot2 = figure(width=400, height=400)
plot2.line("x", "y1", source=source2, line_width=3, line_alpha=0.6)
plot2.line("x", "y2", source=source2, line_width=3, line_alpha=0.6, line_color="orange")

max_val_1 = Slider(value=16.0, start=10.0, end=30.0, step=1.0, title="Response gain 1")
slope_1 = Slider(value=-40, end=-5.0, start=-60.0, step=1.0, title="Input gain 1")
midpoint_1 = Slider(
    value=200.0, start=25.0, end=400.0, step=1.0, title="Current offset 1"
)

callback = CustomJS(
    args=dict(
        source1=source1,
        source2=source2,
        max_val_1=max_val_1,
        slope_1=slope_1,
        midpoint_1=midpoint_1,
    ),
    code="""
    const data = source1.data;
    const x_array = data['x'];
    const y1 = x_array.map((x) => {
        return 1 / (1 + Math.exp((x - midpoint_1.value) / slope_1.value)) * max_val_1.value
    })
    source1.data['y1'] = y1
    const y1_diff = y1.slice(1).map((value, index) => value - y1[index])
    source2.data['y1'] = y1_diff
    source1.change.emit();
    source2.change.emit();
""",
)

max_val_1.js_on_change("value", callback)
slope_1.js_on_change("value", callback)
midpoint_1.js_on_change("value", callback)

show(
    column(
        column(max_val_1, slope_1, midpoint_1),
        gridplot([[plot1, plot2]]),
    )
)

### Fit the data to a sigmoid curve
First let's look at the FI data that we extract from the acquisitions

In [ ]:
spike_analysis = {"acqs": [], "hertz": [], "current": [], "peaks": [], "voltages": []}
time = (pulse_end - pulse_start) / 10000

fi_plot = figure(
    x_axis_label="Current (pA)",
    y_axis_label="Firing rate (Hertz)",
    width=400,
    height=400,
)
for acq_num, acq in exp_dict.items():
    voltages = np.array(acq["array"])
    peaks, _ = signal.find_peaks(
        voltages[pulse_start:pulse_end],
        height=10,
        prominence=10,
    )
    peaks += pulse_start
    spike_analysis = {}
    spike_analysis["hertz"] = len(peaks / time)
    spike_analysis["current"] = acq["pulse_amp"]
    spike_analysis["peaks"] = peaks
    spike_analysis["voltages"] = voltages[peaks]
    acq.update(spike_analysis)

In [ ]:
# Retrieving the current and hertz from our experimental dictionary for ease of use
current = np.array([i["current"] for i in exp_dict.values()])
hertz = np.array([i["hertz"] for i in exp_dict.values()])
fi_plot.scatter(
    current,
    hertz,
    line_color="black",
    color="grey",
    size=10,
    alpha=0.6,
)
show(fi_plot)

Next we are going to curve fit the line. We will use [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). This function needs a function passed to it, in this case it is the sigmoid function as well as an array for the x values and y values. There are many other factors that you could pass, such as bounding factors but are not needed in this case. Lastly you can fit the entire array of current values, however neurons will only spike when there is a net positive current injection. We can truncate our arrays of current and hertz so that only the pairs that have a current >= 0 are kept. You can fit the sigmoid curve to whole set of data and it will likely fit well.

In [ ]:
indexes = np.where(current >= 0)[0]
p, _ = optimize.curve_fit(sigmoid, current[indexes], hertz[indexes])
fit_x = np.linspace(0, max(current), 1000)
fit_y = sigmoid(fit_x, p[0], p[1], p[2], p[3])

fi_plot = figure(
    x_axis_label="Current (pA)",
    y_axis_label="Firing rate (Hertz)",
    width=400,
    height=400,
)
fi_plot.scatter(
    current,
    hertz,
    line_color="black",
    color="grey",
    size=10,
    alpha=0.6,
)
fi_plot.line(fit_x, fit_y, line_width=2)

diff = np.diff(fit_y)
max_gain_index = diff.argmax()
max_gain = diff[max_gain_index]
max_current = fit_x[max_gain_index]

gain_plot = figure(
    x_axis_label="Current (pA)", y_axis_label="Gain (Slope)", width=400, height=400
)
gain_plot.line(fit_x[1:], diff, line_width=2)
dst_end = Span(
    location=max_current, dimension="height", line_color="orange", line_width=2
)
gain_plot.add_layout(dst_end)
print(
    f"Slope (gain): {p[2]}",
    f"Max value: {p[0]}",
    f"Midpoint: {p[1]}",
    f"Max gain: {max_gain}",
    sep="\n",
)
show(gridplot([[fi_plot, gain_plot]]))

### Changes in IEI with frequency
The time between each spike, interevent interval (IEI) or interspike interval (ISI), can also change with spike frequency. Parvalbumin interneurons tend to have a very stable IEI. They can also burst fire a lower current injects with short IEIs followed by a longer IEI then short IEIs. Pyramidal cells on the other can have a lengthing of the IEI the more spikes there are. However, I have noticed in younger mice (P16) that this only occurs when the cell is spiking at higher frequencies.

In [ ]:
iei_fig = figure(
    width=400, height=250, y_axis_label="IEI (ms)", x_axis_label="Number of spikes"
)
peaks = [i["peaks"] for i in exp_dict.values()]
for p in peaks:
    if len(p) > 1:
        x = np.arange(len(p) - 1)
        iei = np.diff(p / 10)
        iei_fig.line(x, iei, color="black")
show(iei_fig)

### Spike adaptation index
Spike adaptation is a way to quantify whether IEI is stable or not. There are many ways to calculate the spike adaptation and we will cover a few of them here.
3. Local spike variability {cite}`shinomoto_relating_2009`

#### Allen institute method
This method is ideal if you want to know if your IEIs are increasing or decreasing since the values will be bound as -1 and 1. A negative value means the spikes are slowing down and positive value means the spikes are speeding up.

In [ ]:
sa = []
npeaks = []
for p in peaks:
    if len(p) > 2:
        iei = np.diff(p)
        if np.allclose((iei[1:] + iei[:-1]), 0.0):
            spike_adapt = np.nan
        norm_diffs = (iei[1:] - iei[:-1]) / (iei[1:] + iei[:-1])
        norm_diffs[(iei[1:] == 0) & (iei[:-1] == 0)] = 0.0
        spike_adapt = np.nanmean(norm_diffs)
        sa.append(spike_adapt)
        npeaks.append(len(p))
iei_fig = figure(
    width=400,
    height=250,
    y_axis_label="Spike adaptation",
    x_axis_label="Number of spikes",
)
iei_fig.scatter(npeaks, sa, size=10, color="grey", line_color="black")
print(np.mean(sa))
show(iei_fig)

#### Coefficient of variation (CV)
This is the simplest method. The coefficient of variation (CV) is the standard deviation of IEIs divided by the mean IEI. The CV measures the dispersion of the data.

In [ ]:
sa = []
npeaks = []
for p in peaks:
    if len(p) > 1:
        iei = np.diff(p)
        sa.append(np.std(iei) / np.mean(iei))
        npeaks.append(len(p))
iei_fig = figure(
    width=400,
    height=250,
    y_axis_label="Spike adaptation",
    x_axis_label="Number of spikes",
)
iei_fig.scatter(npeaks, sa, size=10, color="grey", line_color="black")
print(np.mean(sa))
show(iei_fig)

#### Local variation
Local variation is from Shinomoto et al., 2009 {cite}`shinomoto_relating_2009`. One problem with the CV is that vastly different spike patterns can result in the same CV. Shinomoto developed an equation that get around that issue.

In [ ]:
sa = []
npeaks = []
for p in peaks:
    if len(p) > 2:
        iei = np.diff(p)
        isi_shift = iei[1:]
        isi_cut = iei[:-1]
        n_minus_1 = len(isi_cut)
        local_var = (
            np.sum((3 * (isi_cut - isi_shift) ** 2) / (isi_cut + isi_shift) ** 2)
            / n_minus_1
        )
        sa.append(local_var)
        npeaks.append(len(p))
iei_fig = figure(
    width=400,
    height=250,
    y_axis_label="Spike adaptation",
    x_axis_label="Number of spikes",
)
iei_fig.scatter(npeaks, sa, size=10, color="grey", line_color="black")
print(np.mean(sa))
show(iei_fig)

#### Revised local variation
Revised local variation is from Shinomoto et al., 2009 {cite}`shinomoto_relating_2009`. One problem with the CV is that vastly different spike patterns can result in the same CV. Shinomoto originally developed the local variation equation to fix this issue, however they later adjusted the equation to account for the refactory period of the neuron in being analyzed.

In [ ]:
sa = []
npeaks = []

# Generic refactory periond of 2 ms will work.
R = 2
for p in peaks:
    if len(p) > 2:
        iei = np.diff(p / 10)
        isi_shift = iei[1:]
        isi_cut = iei[:-1]
        isi_add = isi_cut + isi_shift
        left = 1 - ((4 * isi_cut * isi_shift) / isi_add**2)
        right = 1 + ((4 * R) / isi_add)
        var = 3 * np.sum(left * right) * len(isi_shift)
        sa.append(var)
        npeaks.append(len(p))
iei_fig = figure(
    width=400,
    height=250,
    y_axis_label="Spike adaptation",
    x_axis_label="Number of spikes",
)
iei_fig.scatter(npeaks, sa, size=10, color="grey", line_color="black")
print(np.mean(sa))
show(iei_fig)

## Changes in spike amplitude with frequency
Another analysis we can look at is how spike amplitude changes with frequency. Some neurons like interneurons have extremely stable spike amplitude even when spiking at high frequencies where as other cells like pyramidal cells in the frontal cortex do not have a stable spike amplitude. We can get a general idea of whether and how much the peaks are increasing or decreasing in size by taking the derivative between the peaks and averaging the differences.

In [ ]:
voltages = [i["voltages"] for i in exp_dict.values()]
amp_fig = figure(
    width=400, height=250, y_axis_label="Voltage (mV)", x_axis_label="Number of spikes"
)
for v in voltages:
    if len(v) > 0:
        x = np.arange(len(v))
        amp_fig.line(x, v, color="black")
show(amp_fig)

sa = []
for p in voltages:
    if len(p) > 1:
        iei = np.diff(p)
        sa.append(np.mean(iei))
print(f"Est slope of peak change over spike number: {np.mean(sa)}")

## Spike threshold
Spike threshold is tells you at what voltage a cells spike is elicited. This different from rheobase. Spike threhold is primarly set by the number of sodium channels on the axon hillock. You can measure spike threshold on any acquisition that has a spike, however usually just the spikes from the rheobase acquisitions are used to calculate the spike threshold. There are several ways to calculate the spike threshold such a the first derivative, second derivative, third derivative, and a max curvature {cite}`sekerli_estimating_2004`. I find the third derivative and method VII to be the most consistent. We will use the third derivative method in this tutorial. The way I typically differentiate signals is by using [numpy.gradient](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html). Unlike [numpy.diff](https://numpy.org/doc/stable/reference/generated/numpy.diff.html), [numpy.gradient](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html) returns an array of the same length as the input signal which makes downstream analysis easier. It has the downside in that if you have small peaks you will get some artifacts in the signal which upsampling can counteract. One problem I found with the third derivative is that the peaks found a usually to late by 1 sample at 10000 Hz so I adjust the spike threshold backwards by 1 sample. I also z-score the derivative since this keeps the peak finding threshold consistent between acquisitions.

First lets look a single acquisition and see how the different derivatives look. One thing we are going to do just for visualization purposes is limit the extent of the x-axis so that we are just looking at the first spike and for a point of reference we are going to use the spike peak. If you look at the third derivative we want to extract the first positive peak. There are some additions that I have found are helpful.

In [ ]:
voltages = np.array(exp_dict[rheobase_acq[0]]["array"])
peak = exp_dict[rheobase_acq[0]]["peaks"][0] - pulse_start

# grab the voltages just inside the current pulse
voltages = voltages[pulse_start:pulse_end]
x = np.arange(voltages.size)

# First derivative
dv = np.gradient(voltages)
# Second derivative
ddv = np.gradient(dv)
# Third derivateive
dddv = np.gradient(ddv)

source = ColumnDataSource({"x": x, "v": voltages, "dv": dv, "ddv": ddv, "dddv": dddv})
dst_end = Span(location=peak, dimension="height", line_color="orange", line_width=2)

vplot = figure(title="Voltage (mV)", width=300, height=200, x_range=(1120, 1220))
vplot.line(x="x", y="v", source=source)
vplot.add_layout(dst_end)
dvplot = figure(title="dV", width=300, height=200, x_range=vplot.x_range)
dvplot.line(x="x", y="dv", source=source)
dvplot.add_layout(dst_end)
ddvplot = figure(title="ddV", width=300, height=200, x_range=vplot.x_range)
ddvplot.line(x="x", y="ddv", source=source)
ddvplot.add_layout(dst_end)
dddvplot = figure(title="dddV", width=300, height=200, x_range=vplot.x_range)
dddvplot.line(x="x", y="dddv", source=source)
dddvplot.add_layout(dst_end)
grid = gridplot([[vplot, dvplot], [ddvplot, dddvplot]])
show(grid)

We are going to calculate the spike threshold for every spike in an acquisition that contains spikes. We will use the third derivative method. To do this we will create two functions. One to find a spike threshold for a single spike in a window. The second function will loop through all of the spikes to find the spike threshold using a different start and stop values for each spike. 

In [ ]:
def find_spk_threshold(voltages: np.array, start: int, end: int):
    dv = np.gradient(voltages)
    ddv = np.gradient(dv)
    dddv = np.gradient(ddv)

    temp = dddv[start:end]
    base = temp.argmin()
    index = base - 1
    val = temp[base] - temp[index]
    while val < 0:
        index -= 1
        base -= 1
        val = temp[base] - temp[index]
    thresh_peak = start + index
    return thresh_peak, voltages[thresh_peak]


def find_all_spk_thresholds(voltages, peaks, pulse_start):
    output = np.zeros((len(peaks), 2))
    start_index = pulse_start
    for index in range(len(peaks)):
        if index < (len(peaks) - 1):
            end_index = peaks[index]
        else:
            end_index = pulse_end
        output[index] = find_spk_threshold(voltages, start_index, end_index)
        start_index = peaks[index]
    return output


for acq in exp_dict.values():
    voltages = np.array(acq["array"])
    peaks = acq["peaks"]
    if len(peaks) > 0:
        output = find_all_spk_thresholds(voltages, peaks, pulse_start)
        acq["spike_threshold"] = output
    else:
        acq["spike_threshold"] = None

To calculate the spike threshold we are only going to get the spike threshold for the first spike of each rheobase acquisition. This is pretty common practice.

In [ ]:
spike_threshold_x = []
spike_threshold_y = []
for i in rheobase_acq:
    spike_threshold_y.append(exp_dict[i]["spike_threshold"][0, 1])
    spike_threshold_x.append(exp_dict[i]["spike_threshold"][0, 0])

print(f"Spike threshold: {np.mean(spike_threshold_y):.3f} mV")

Let's inspect the output. While it may look like the spike threshold is climbing up the spike a little bit, it is important to remember what a failed spike looks like. There is often a hump suggesting there is a small and rapid climb in voltage before a spike can occur even if the spike does not occur. We are trying to find that bifurcation point. Unfortunately, I did not include an acquisition with a failed spike to show the difference.

In [ ]:
figures = []
temp = {str(i): exp_dict[i]["array"] for i in rheobase_acq}
temp["x"] = np.arange(len(temp[str(rheobase_acq[0])])) / 10
source = ColumnDataSource(data=temp)
for sx, sy, acq_num in zip(spike_threshold_x, spike_threshold_y, rheobase_acq):
    sx = sx / 10
    vplot = figure(
        x_axis_label="Time (ms)",
        y_axis_label="Voltage (mV)",
        width=300,
        height=200,
        x_range=(sx - 5, sx + 10),
        y_range=(-50, 40),
    )
    vplot.line(x="x", y=str(acq_num), source=source, color="black")
    vplot.scatter(sx, sy, size=10, color="orange")
    figures.append(vplot)
show(gridplot([figures[:2], figures[2:]]))

We will also look at how the spike threshold changes as the number of spikes increases and at the time the spike occurs.

In [ ]:
thres_n = figure(
    width=400,
    height=250,
    y_axis_label="Spike threshold (mV)",
    x_axis_label="Spike number",
)
thres_time = figure(
    width=400,
    height=250,
    y_axis_label="Spike threshold (mV)",
    x_axis_label="Spike time (ms)",
)
thr = [i["spike_threshold"] for i in exp_dict.values()]
for p in thr:
    if p is not None:
        x = np.arange(p.shape[0]) + 1
        thres_n.line(x, p[:, 1], color="black")
        thres_time.line(p[:, 0] / 10, p[:, 1], color="black")
show(row(thres_n, thres_time))

## Spike (half) width
The next waveform feature that we can measure is the spike (half) width. Half width is important because it can help you determine if there is broadening or shortening of the waveform. Broadening of the waveform can be due to inactivation of the voltage-gated potassium channels, slower-inactivating sodium channels or even decreased activation of voltage-gated calcium channels. Wavefrom broadening increases synaptic output by increasing release probability by increasing the time calcium is in the presynapse {cite}`zbili_past_2019`. This is where the voltage crosses half way between the spike threshold and peak. One important caveat of measuring spike width as the half-way point is that if your spike threshold or peak is changed then your half-width will likely be different.

In [ ]:
def find_spk_width(
    voltages: np.array, peak_x: int, spike_threshold: float, start: int, end: int
):
    volts = np.asarray(voltages[int(start) : int(end)])
    masked_array = volts.copy()
    mask = np.array(volts > spike_threshold)
    masked_array[~mask] = spike_threshold
    width = signal.peak_widths(masked_array, [int(peak_x - start)], rel_height=0.5)
    width[2][0] += start
    width[3][0] += start
    width = np.array([width[2][0], width[3][0], width[1][0]])
    return width


def find_all_spk_widths(voltages, peaks, spike_thresholds, pulse_end):
    output = np.zeros((len(peaks), 3))
    for index in range(len(peaks)):
        if index < (len(peaks) - 1):
            end = spike_thresholds[index + 1, 0]
        else:
            # Adding 2000 samples (or 2 ms) to the end helps with spikes that occur just before the end of the acquisition
            if (pulse_end - spike_thresholds[index, 0]) < 2000:
                end = spike_thresholds[index, 0] + 2000
            else:
                end = pulse_end
        output[index] = find_spk_width(
            voltages,
            peaks[index],
            spike_thresholds[index, 1],
            spike_thresholds[index, 0],
            end,
        )
    return output

In [ ]:
spike_width_x = []
spike_width_y = []
for acq in exp_dict.values():
    voltages = acq["array"]
    spk_thresholds = acq["spike_threshold"]
    peaks = acq["peaks"]
    if len(peaks) > 0:
        acq["spike_widths"] = find_all_spk_widths(
            voltages, peaks, spk_thresholds, pulse_end
        )
    else:
        acq["spike_widths"] = None

To calculate the spike width we are only going to get the spike width for the first spike of each rheobase acquisition. This is pretty common practice.

In [ ]:
spike_width = []
for i in rheobase_acq:
    spike_width.append(
        (exp_dict[i]["spike_widths"][0, 1] - exp_dict[i]["spike_widths"][0, 0]) / 10
    )

print(f"Spike width: {np.mean(spike_width):.3f} ms")

Let's inspect the spike width measurements to make sure they are correct.

In [ ]:
figures = []
for acq_num in rheobase_acq:
    acq = exp_dict[acq_num]
    sx = acq["spike_widths"][0, :2]
    sy = acq["spike_widths"][0, 2]
    sy = [sy, sy]
    vplot = figure(
        width=300,
        height=200,
        x_range=(sx[0] / 10 - 5, sx[0] / 10 + 10),
        y_range=(-50, 40),
        x_axis_label="Time (ms)",
        y_axis_label="Voltage (mV)",
    )
    vplot.line(x="x", y=str(acq_num), source=source, color="black")
    vplot.line(np.array(sx) / 10, sy, line_width=3, color="orange")
    figures.append(vplot)
show(gridplot([figures[:2], figures[2:]]))

Similar to spike threshold we can look at spike width as the number of spikes increases. Spike width in this cell shows logarithmic growth. We will also fit a curve to the cell's data. Note that we are fitting the spike number as x and not the time when the spike occurs.

In [ ]:
def log_func(x, vscale, hscale, offset):
    return vscale * np.log(hscale * x) + offset


spk_width_n = figure(
    width=400, height=250, y_axis_label="Spike width (ms)", x_axis_label="Spike number"
)
spk_width_time = figure(
    width=400,
    height=250,
    y_axis_label="Spike width (ms)",
    x_axis_label="Spike time (ms)",
)
width = [i["spike_widths"] for i in exp_dict.values()]

for p in width:
    if p is not None:
        y = (p[:, 1] - p[:, 0]) / 10
        time = p[:, 0] / 10
        spk_width_time.line(time, y, color="black")
        spk_width_n.line(np.arange(time.size) + 1, y, color="black")

xs = []
ys = []
for p in width:
    if p is not None:
        ys.extend((p[:, 1] - p[:, 0]) / 10)
        xs.extend(np.arange(p.shape[0]) + 1)
popt, pcov = optimize.curve_fit(log_func, xs, ys)
x_fit = np.linspace(min(xs), max(xs), num=100)
y_fit = log_func(x_fit, *popt)
spk_width_n.line(x_fit, y_fit, color="orange", width=3)

show(row(spk_width_n, spk_width_time))

## Spike afterhyperpolarization (AHP)
The spike afterhyperpolarization (AHP) is the refactory period, the period where a neuron cannot fire another action potential. The AHP is defined as when the membrane voltage drops below the resting membrane potential of a neuron. However, when injecting current. you will rarely see the AHP drop below the resting membrane potential or the potential where the cell is being held. The are a a couple ways that you could analyze the AHP. One way is setting a time cutoff after the spike for what part of the AHP you want to analyze and integrate over that period. One thing to consider with this method is that you will likely want to integrate over a percent of the AHP rather than using a strict time cutoff since the AHP will get shorter at higher spike frequencies. This is typically how people calculate the slow and fast components of the AHP. You can also find the peak or most negative component of the AHP and when it occurs.

### Measuring the peak AHP

In [ ]:
def find_all_ahp_peaks(voltages, peaks, pulse_end):
    output = np.zeros((len(peaks), 2))
    for index in range(len(peaks)):
        if index < (len(peaks) - 1):
            t = np.argmin(voltages[peaks[index] : peaks[index + 1]]) + peaks[index]
            v = voltages[t]
        else:
            t = np.argmin(voltages[peaks[index] : pulse_end]) + peaks[index]
            v = voltages[t]
        output[index] = [t, v]
    return output


for acq in exp_dict.values():
    voltages = acq["array"]
    peaks = acq["peaks"]
    if len(peaks) > 0:
        acq["ahp"] = find_all_ahp_peaks(voltages, peaks, pulse_end)
    else:
        acq["ahp"] = None

In [ ]:
spike_ahp_time = []
spike_ahp_volt = []
for i in rheobase_acq:
    spike_ahp_time.append(exp_dict[i]["ahp"][0, 0] / 10)
    spike_ahp_volt.append(exp_dict[i]["ahp"][0, 1])
print(f"AHP time: {np.mean(spike_ahp_time):.3f} ms")
print(f"AHP volt: {np.mean(spike_ahp_volt):.3f} mV")

In [ ]:
figures = []
for acq_num in rheobase_acq:
    acq = exp_dict[acq_num]
    sx = acq["ahp"][0, 0] / 10
    sy = acq["ahp"][0, 1]
    vplot = figure(
        width=300,
        height=200,
        x_range=(sx - 100, sx + 200),
        y_range=(-60, 45),
        x_axis_label="Time (ms)",
        y_axis_label="Voltage (mV)",
    )
    vplot.line(x="x", y=str(acq_num), source=source, color="black")
    vplot.scatter(sx, sy, size=10, color="orange")
    figures.append(vplot)
show(gridplot([figures[:2], figures[2:]]))

We can look at the AHP as the number of spikes increases. One thing you will notice is that the the last AHP on several acquisitions is much higher than the rest of the AHPs. This is because those spikes are truncated by the end of the current injection. You can filter those out if you want or leave them in.

In [ ]:
spk_ahp_n = figure(
    width=400, height=250, y_axis_label="Voltage (mV)", x_axis_label="Spike number"
)
spk_ahp_time = figure(
    width=400,
    height=250,
    y_axis_label="Spike width (ms)",
    x_axis_label="Spike time (ms)",
)
width = [i["ahp"] for i in exp_dict.values()]
for w in width:
    if w is not None:
        time = w[:, 0] / 10
        y = w[:, 1]
        spk_ahp_time.line(time, y, color="black")
        spk_ahp_n.line(np.arange(time.size) + 1, y, color="black")
show(row(spk_ahp_n, spk_ahp_time))

## Spike derivative
The spike derivative is useful because we can look at the currents underlying the spike itself. While we cannot see the exact Na+, K+ and Ca+ currents we can get an idea of what currents might be altered. To convert a voltage spike to current spike we must multiply the spike derivative by the capacitance of the cell since the current (I) = -C*dV/dt. The current action potential is very similar to the current action potentials that you get when you record spontaneous action potentials is cell-attached voltage-clamp mode. This equation works because a cell is a capacitor due to the lipid membrane. The first peak, or most negative peak primarily consists of Na+ currents where as the second peak likely consists of K+ and Ca+ currents {cite}`bean_action_2007`. One interesting thing to note is the peak current in these spikes (~-2000 pA) is typically around the current you get with unclamped spikes when running a electrical/optical stimulation experiment.

In [ ]:
first_spikes = []
offset = int(0.5*10)
for acq in rheobase_acq:
    array = exp_dict[acq]["array"]
    start = int(exp_dict[acq]["spike_threshold"][0,0])-offset
    if exp_dict[acq]["spike_threshold"].shape[0] > 1:
        end = int(exp_dict[acq]["spike_threshold"][1,0])
    else:
        peak_x = exp_dict[acq]["peaks"][0]
        thresh = exp_dict[acq]["spike_threshold"][0,1]
        ind = np.where(array[peak_x:] < thresh)[0]+peak_x
        end = min(ind[-1], pulse_end)
    first_spikes.append(array[start:end])

min_len = min([len(i) for i in first_spikes])
first_spikes = [i[:min_len] for i in first_spikes]
derivative = [np.gradient(i)*-85 for i in first_spikes]
peak_na = np.mean([min(i) for i in derivative])
peak_k_ca = np.mean([max(i) for i in derivative])
print(f"Peak Na+: {peak_na} pA; Peak K+/Ca2+: {peak_k_ca} pA")

In [ ]:
fig1 = figure(height=250, width=300, y_axis_label="mV")
fig2 = figure(height=250, width=300, x_range=(0,40), y_axis_label="pA")
fig3 = figure(height=250, width=300)
for i, j in zip(first_spikes, derivative):
    x = np.arange(len(i))
    fig1.line(x, i, color="black")
    fig2.line(x, j, color="black")
    fig3.line(i, -j, color="black")
show(row(fig1, fig2, fig3))

## Rebound spikes
Rebound spikes typically occur after a hyperpolarizing pulse. Dopaminergic cells typically have a rebound spike. I have not seen any analyses of the rebound spike(s), however this is feature that your cell could have and you could note the presence or number of the rebound spike(s).

In [ ]:
p = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/rebound_spike/1.json"
with urllib.request.urlopen(p) as url:
    file = json.load(url)
array = file["array"]
rs = figure(height=250, width=400)
rs.line(np.arange(len(array)), array, color="black")
show(rs)

```{bibliography}
:filter: docname in docnames
```